In [1]:
#Data source
dpath = "/home/ubuntu/data/"
import pandas as pd, numpy as np

#Preprocessing steps
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

#Metrics to use
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score
beta = 2 #favors recall 
from sklearn.model_selection import cross_val_score

#Classifiers
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [2]:
from __future__ import print_function

In [3]:
#List of all classifiers that will be trained
classifiers = [LogisticRegression(n_jobs=-1), RandomForestClassifier(n_jobs=-1), GaussianNB(), 
               MLPClassifier(), KNeighborsClassifier(n_jobs=-1), SVC(probability=True)]

In [4]:
#List of all features that will be combined
feat_to_combine = ["AutoColorCorrelogram", "BinaryPatternsPyramid", "EdgeHistogram", "FCTH", "FuzzyColorHistogram", 
                       "FuzzyOpponentHistogram", "Gabor", "JCD", "JpegCoefficientHistogram", "LocalBinaryPatterns", 
                       "LuminanceLayout", "OpponentHistogram", "PHOG", "RotationInvariantLocalBinaryPatterns", "ScalableColor", 
                       "SimpleColorHistogram", "Tamura", "CEDD", "ColorLayout"]

In [5]:
#Loading of the train and test set descriptors
train_set = pd.read_csv(dpath + "train_set.csv")
test_set = pd.read_csv(dpath + "test_set.csv")

In [6]:
features_train = train_set[["Image"]].set_index("Image")
features_test = test_set[["Image"]].set_index("Image")

In [7]:
#Combining all the features

for feat in feat_to_combine:
    f1_train = pd.read_csv(dpath + "features_%s_%s.csv" % (feat, "train"), header=None, skiprows=1)
    f1_train = f1_train.drop(f1_train.columns[0], axis=1)
    f1_train = f1_train.set_index(train_set["Image"])
    f1_test = pd.read_csv(dpath + "features_%s_%s.csv" % (feat, "test"), header=None, skiprows=1) 
    f1_test = f1_test.drop(f1_test.columns[0], axis=1)
    f1_test = f1_test.set_index(test_set["Image"])
    
    features_train = pd.concat([features_train, f1_train], axis=1)
    features_test = pd.concat([features_test, f1_test], axis=1)

In [8]:
#Function to obtain the classifier name from their string representation
def classifier_name(classifier):
    return str(classifier).split("(")[0]

In [9]:
print("Shape of train features", features_train.as_matrix().shape)
print("Shape of test features", features_test.as_matrix().shape)

Shape of train features (2000, 3778)
Shape of test features (2000, 3778)


In [10]:
#ids and targets

id_train = train_set["Image"]
y_train = train_set["Class"]

id_test = test_set["Image"]
y_test = test_set["Class"]

In [11]:
%%time

metrics = pd.DataFrame(columns=["model", "accuracy", "F1" ,"precision", "recall"])
    
#Feature transformation pipeline: MinMaxScaling
processing_pipeline = make_pipeline(MinMaxScaler())
X_features_train = processing_pipeline.fit_transform(features_train.as_matrix())
X_features_test = processing_pipeline.transform(features_test.as_matrix())
    
#Training classifiers with default parameters
for algo in classifiers:
    algo_name = classifier_name(algo)
    print("Training", algo_name)
    algo = algo.fit(X_features_train, y_train)
    print("predicting")
    y_pred = algo.predict(X_features_test)
    m = {"model": algo_name, 
            "accuracy": accuracy_score(y_test, y_pred), 
            "F1": f1_score(y_test, y_pred, average='macro'), 
            "precision": precision_score(y_test, y_pred, average='macro'), 
            "recall": recall_score(y_test, y_pred, average='macro'),
            "Fbeta": fbeta_score(y_test, y_pred, beta=beta, average='macro') 
        }
    metrics = metrics.append(m, ignore_index=True)

Training LogisticRegression
predicting
Training RandomForestClassifier
predicting
Training GaussianNB
predicting
Training MLPClassifier
predicting
Training KNeighborsClassifier
predicting
Training SVC
predicting
CPU times: user 4min 1s, sys: 32.1 s, total: 4min 33s
Wall time: 2min 35s


In [12]:
metrics

model  accuracy        F1  precision  recall     Fbeta
0      LogisticRegression    0.8700  0.869670   0.870228  0.8700  0.869762
1  RandomForestClassifier    0.7490  0.747252   0.751454  0.7490  0.747598
2              GaussianNB    0.5645  0.544855   0.565031  0.5645  0.551990
3           MLPClassifier    0.8630  0.862813   0.864514  0.8630  0.862698
4    KNeighborsClassifier    0.6830  0.674100   0.699104  0.6830  0.676023
5                     SVC    0.7770  0.775388   0.779253  0.7770  0.775722

In [13]:
metrics.sort_values(by=["accuracy", "F1"], ascending=False).to_csv(dpath+"metrics_early_fusion_Set2.csv")